# Francony Alexandre

# WebScrapper for MTG cards on cardmarket.com

![img infos](images/cardmarket.png "Logo Cardmarket")

## Objectif : 

### 1) Récupérer toute les cartes de l'extensions "Phyrexia : all wil be one" sur le site cardmarket.com
#### URL : https://www.cardmarket.com/en/Magic/Products/Singles/Phyrexia-All-Will-Be-One

### 2) Récupérer les informations suivantes :
#### - Nom de la carte
#### - 5 prix de vente (les plus bas)
#### - La quantité d'exemplaire disponible
#### - Le prix moyen de vente sur 7 jours
#### - Une image de la carte

Nous allons dans un premier temps nous assurer que le site est atteignable à l'aide de la librairie requests et de la fonction get_status_code. Cette fonction sera réutilisée dans le code pour vérifier que le site est toujours atteignable avant chaque étape de récupération majeure.

In [1]:
import requests
import csv
import os
import requests
from bs4 import BeautifulSoup
import re

def get_status_code(url):
    response = requests.get(url)
    return response.status_code

url = "https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One"
status_code = get_status_code(url)
print("Le code d'état de Cardmarket est", status_code)


Le code d'état de Cardmarket est 200


Nous allons commencer par créer quelques fonctions permettant de récupérer les informations sur le dictionnaire contenant les cartes récupérés, c'est ce dernier qui contiendra toute les informations sur les cartes récupérés, il est donc vital de s'assurer de sa bonne structure. Prep () nous servirera à transformer le nom d'une carte en son lien afin d'accéder aux infos.

In [2]:
def head(dict):# Fonction qui permet d'afficher les 5 premiers éléments d'un dictionnaire
    i = 0
    for key in dict:
        if(i < 5):
            print(key, dict[key])
            i += 1
        else:
            break

def foot(dict):# Fonction qui permet d'afficher les 5 derniers éléments d'un dictionnaire
    i = 0
    for key in reversed(dict):
        if(i < 5):
            print(key, dict[key])
            i += 1
        else:
            break

def info(dicto):
    print(f"{len(dicto)} cartes récupérées.\n")
    print("5 premières cartes :")
    head(dicto)
    print("\n5 dernières cartes :")
    foot(dicto)

A présent que nous avons vérifier la disponibilité du site, nous allons pouvoir commencer à récupérer les données.

Nous voyons donc via la sortie de cette cellule que nous avons bien récupéré les données de la première page en montrant le nombre de cartes récupérées, les 5 premières cartes et les 5 dernières cartes.

In [3]:
# URL de la page de l'extension Phyrexia All will be one
url_template = "https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One?sortBy=price_desc&site={page}"

if(get_status_code(url_template.format(page=1)) == 200):
    # Création d'un dictionnaire pour stocker les noms de cartes et leurs URL correspondantes
    card_url = {}
    cards_divs = []
    
    for page in range(1, 15):
        url = url_template.format(page=page)

        # Récupération du contenu de la page
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Recherche de la liste des cartes
        table = soup.find_all('div', {'class': 'row no-gutters'})

        cards_list = []

        for row in table:
            cards_list.append(row)

        for card in cards_list:
            card_name = card.find('div', {'class': 'col-10 col-md-8 px-2 flex-column align-items-start justify-content-center'}).find('a').text.strip()
            card_url[card_name] = "https://www.cardmarket.com" + card.find('div', {'class': 'col-10 col-md-8 px-2 flex-column align-items-start justify-content-center'}).find('a').get('href')

    # Affichage du nombre de cartes récupérées
    info(dicto = card_url)

else:
    print("Le code d'état de Cardmarket est", status_code, ", la page n'a pas pu être chargée.")


160 cartes récupérées.

5 premières cartes :
Elesh Norn, Mother of Machines https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Elesh-Norn-Mother-of-Machines
Atraxa, Grand Unifier https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Atraxa-Grand-Unifier
Mondrak, Glory Dominus https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Mondrak-Glory-Dominus
Sword of Forge and Frontier https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Sword-of-Forge-and-Frontier
Oblitérateur phyrexian https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Phyrexian-Obliterator

5 dernières cartes :
Voile d'assimilation https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Veil-of-Assimilation
Dévotion parachevée https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One/Compleat-Devotion
Bombe crânienne du Fourneau https://www.cardmarket.com/fr

Nous allons donc créer une fonction par type de données que nous souhaitons récupérer, le nom, le prix des 5 cartes les moins cher, la quantité de cartes en ventes, le prix moyen sur 7 jours, ainsi qu'une image de la carte. 

In [4]:
def getCardName(card):
    #récupérer le nom de la carte qui est la clé du dictionnaire
    card_name = card
    if(card_name == ''):
        card_name = 'Undefined'
    return card_name;

def getCardPrices(card):
    cards_sales = []
    tab_prices = []
    
    # Récupération du contenu de la page
    url = card_url[card]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Recherche de la liste des cartes
    table = soup.find_all('div', {'class': 'row no-gutters article-row'})

    for row in table:
        cards_sales.append(row)

    #récupérer les 5 premiers prix de vente de la carte
    for card in cards_sales:
        card_price = card.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2]
        tab_prices.append(card_price.replace(',', '.'))
        if len(tab_prices) == 5:
            break

    return tab_prices;

def getCardQuantity(card):
    # récupérer l'url de la carte
    url = card_url[card]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération de la quantité de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]

    # récupérer seulement la quantité, qui est contenu dans le premier élément du tableau
    card_quantity = frame_infos[0].text.strip()

    return card_quantity;

def getCardAveragePrice(card):
    # Récupération du contenu de la page
    url = card_url[card]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération des données en chiffre de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]
    
    # récupérer seulement le prix moyen sur 7 jours, qui est contenu dans l'avant dernier élément du tableau
    card_average_price = frame_infos[-2].text.strip()
    return card_average_price.replace(',', '.').replace(' €', '').replace('£', '');

def getCardImage(card):
    # Récupération du contenu de la page
    url = card_url[card]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération de l'image de la carte
    card_image = soup.find('img', {"class" : "is-front", "alt" : card}).get('src')
    
    # enlever les 2 premier caractères de l'url de l'image (//)
    card_image = card_image[2:]
    return card_image;

La fonction si dessous nous permet donc d'exploiter toute nos fonctions getX pour récupérer les données sur les cartes et les mettre dans un dictionnaire card_intel. cela nous permettra de récupérer les informations sur les cartes plus facilement grâce à l'appel de cette fonction.

In [5]:
def getCardInfo(card):
    url = card_url[card]
    if(get_status_code(url) == 200):
        card_intel = []
        card_intel.append(getCardName(card))
        card_intel.append(getCardPrices(card))
        card_intel.append(getCardQuantity(card))
        card_intel.append(getCardAveragePrice(card))
        card_intel.append(getCardImage(card))
    else:
        print("Le code d'état de la page de ", card, " est ", get_status_code(url), ", la page n'a pas pu être chargée.")
    return card_intel

#for card in card_url:
#    print(getCardInfo(card))

A présent que nous avons récupéré les données, nous allons pouvoir les stocker dans un fichier.csv, j'ai pour cela utilisé la librairie csv pour gérer le fichier de sauvegarde, et la librairie os pour vérifier l'existence du fichier et créer le fichier/dossier s'il n'existe pas.

Une fois le fichier créé, nous allons pouvoir récupérer tour à tour les données de toutes les cartes de l'extension Phyrexia all will be one, et les écrire dans le fichier.

In [6]:
# à décommenter pour créer le fichier .csv (prends environ 1h20 max)

## Création du dossier s'il n'existe pas déjà
#if not os.path.exists('data'):
#    os.makedirs('data')
#
## Création du fichier .csv
#with open('data/cards.csv', 'w', newline='', encoding='utf-8') as csvfile:
#    writer = csv.writer(csvfile, delimiter=';')
#    # nommer les collonnes du tableau
#    writer.writerow(['Card Name', 'Prices', 'Quantity', 'Average Price (7 days)', 'link to picture'])
#    for card in card_url:
#        writer.writerow(getCardInfo(card))

Nous avons donc récupéré les données de toutes les cartes de l'extension Phyrexia all will be one, et les avons stocké dans un data/cards.csv.